In [1]:
import os,sys
import time,datetime
import GPy
import GPyOpt
import numpy as np
from sklearn.metrics import mean_squared_error
from math import sqrt
from scipy.special import expit
import pandas as pd

In [2]:
%run ../tools/TrainTest.py
%run ../tools/PCG1GM.py

In [3]:
# path information
trial_num = 5
tune_trial_num = 100
snapshot_path = '../result/snapshot/complement'
snapshot_form = 'PCG1-hyper6-EI5000w500-split{0}-{1}'
peer_review_csv = '../dataset/open_peer_review/peer_review/peer_review_forPG3_suffled.csv'

# functions
def timeStamp():
    # return time as str
    todaydetail = datetime.datetime.today()
    return todaydetail.strftime("%Y%m%d%H%M%S")

def saveGPyOpt(myBopt,fold_num):
    filename = snapshot_form.format(fold_num,timeStamp())
    path = os.path.join(snapshot_path, filename)
    np.savez_compressed(path, myBopt.X, myBopt.Y)
    print("Save: {}".format(path))

In [4]:
# preprocess train&test dataset
rDF = pd.read_csv(peer_review_csv)
np.random.seed(12345678)
shuffled_pattern = [np.random.permutation(len(rDF)) for i in xrange(trial_num)]
df = rDF.take(shuffled_pattern[0]).reset_index(drop=True)
train, val, test = splitDataset523(df)

In [5]:
pcg1gm = PCG1GM(train, val, test)
pcg1gm.rmseForVal([2.54785885, 4.62864443, 67.68716059, 22.95702442, 6.96275866, 56.26711153])

In [7]:
pcg1gm.rmseForTest([2.54785885, 4.62864443, 67.68716059, 22.95702442, 6.96275866, 56.26711153])

0.7764230031564122

In [6]:
pcg1gm.aucForVal([2.54785885, 4.62864443, 67.68716059, 22.95702442, 6.96275866, 56.26711153])

0.69818902263828919

### ↓making rmseForVal(self,hyper_list), aucForVal(self,hyper_list)

In [8]:
stan_fit = pcg1gm.fit([2.54785885, 4.62864443, 67.68716059, 22.95702442, 6.96275866, 56.26711153])

In [9]:
stan_fit

Inference for Stan model: anon_model_3a76ea8205efb256e97b2f109e97cd41.
4 chains, each with iter=5000; warmup=500; thin=1; 
post-warmup draws per chain=4500, total post-warmup draws=18000.

                   mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
ability[0]          2.7  1.4e-3   0.18   2.35   2.58    2.7   2.82   3.06  18000    1.0
ability[1]         2.55  1.6e-3   0.21   2.13    2.4   2.55   2.69   2.97  18000    1.0
ability[2]         2.78  1.5e-3    0.2   2.39   2.65   2.79   2.92   3.17  18000    1.0
ability[3]         2.68  8.2e-4   0.11   2.46    2.6   2.68   2.75    2.9  18000    1.0
ability[4]         2.42  1.4e-3   0.19   2.05   2.29   2.42   2.55    2.8  18000    1.0
ability[5]         2.93  1.2e-3   0.17    2.6   2.81   2.92   3.04   3.25  18000    1.0
ability[6]         2.76  1.4e-3   0.19   2.39   2.64   2.77   2.89   3.13  18000    1.0
ability[7]         2.55  1.6e-3   0.22   2.12    2.4   2.55   2.69   2.97  18000    1.0
ability[8]         

In [10]:
eap_value = stan_fit.summary()['summary'][:,0]

In [11]:
split = pcg1gm.userNum
ability = eap_value[0:split]
reliability = eap_value[split:split*2]
bias = eap_value[split*2:split*3]
noise = eap_value[split*3]

In [12]:
noise

-1.9988032554678461

In [13]:
sender = pcg1gm.senderVal
receiver = pcg1gm.receiverVal

In [14]:
sender[0], receiver[0]

(96, 12)

In [15]:
val

,sender_id,receiver_id,corrected,value
0,96,12,1,3
1,247,151,0,3
2,30,192,0,3
3,21,67,0,4
4,3,227,1,3
5,57,34,0,4
6,165,200,1,1
7,170,140,1,3
8,247,376,1,3
9,34,269,1,1


In [16]:
sender

array([ 96, 247,  30, ..., 123,  34, 101])

In [17]:
valueEst = ability[receiver] + bias[sender]

In [18]:
from sklearn.metrics import mean_squared_error
from math import sqrt

rmse = sqrt(mean_squared_error(pcg1gm.valueVal, valueEst))

In [19]:
rmse

0.7725239611753643

In [20]:
correctedEst = expit(ability[receiver]+bias[sender]+noise)

In [22]:
from sklearn import metrics

fpr, tpr, thresholds = metrics.roc_curve(pcg1gm.correctedVal, correctedEst, pos_label=1)
auc = metrics.auc(fpr, tpr)

In [23]:
auc

0.69801748796040974

In [25]:
len(pcg1gm.correctedVal), len(correctedEst)

(1050, 1050)